In [1]:
question = "ขับรถชนแล้วหนี มีโทษอะไร"
set_index = "hybrid_pdf_index"

In [2]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding_model_name = 'BAAI/bge-m3'
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=512, device=device)

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
dim = 1024

In [6]:
from os import getenv
from llama_index.vector_stores.opensearch import (
    OpensearchVectorStore,
    OpensearchVectorClient,
)

# http endpoint for your cluster (opensearch required for vector index usage)
endpoint = getenv("OPENSEARCH_ENDPOINT", "http://localhost:9200")
# index to demonstrate the VectorStore impl
idx = getenv("OPENSEARCH_INDEX", set_index)

# OpensearchVectorClient stores text in this field by default
text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled with hybrid search pipeline
client = OpensearchVectorClient(
    endpoint=endpoint,
    index=idx,
    dim=dim,
    embedding_field=embedding_field,
    text_field=text_field,
    search_pipeline="hybrid-search-pipeline",
)

# initialize vector store
vector_store = OpensearchVectorStore(client)

In [7]:
idx

'hybrid_pdf_index'

In [8]:
from llama_index.core import Settings

Settings.llm = None
Settings.embed_model = embed_model

LLM is explicitly disabled. Using MockLLM.


In [9]:
from llama_index.vector_stores.opensearch import OpensearchVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import Settings
from llama_index.core.indices.prompt_helper import PromptHelper

# สร้าง storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# สร้าง index โดยไม่ต้องใช้ prompt_helper ที่กำหนดเอง
index = VectorStoreIndex.from_vector_store(
    vector_store,
    storage_context=storage_context,
)

In [10]:
#! curl -XGET http://localhost:9200/doc_hybrid_pdf_index/_mapping | jq .

In [11]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode
retriever = index.as_retriever(similarity_top_k=3,vector_store_query_mode=VectorStoreQueryMode.HYBRID)
text_retriveve = question
prompt = retriever.retrieve(text_retriveve)
for r in prompt:
    print(r.metadata)
    print(r)

{'page_label': '8', 'file_name': 'article_20160531140556.pdf', 'file_path': '/Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf', 'file_type': 'application/pdf', 'file_size': 301354, 'creation_date': '2024-07-29', 'last_modified_date': '2024-07-28'}
Node ID: 86962834-aa22-4175-a6a8-5c6462d1c7d4
Text: ข้อหา หนึ่งรวมเป็น ๒ ข้อหา ความผิดฐานชนแล้วหนีมีหลักเกณฑ์ดังนี้
เมื่อขับรถหรือขี่หรือควบคุมสัตว์แล้วไปท าให้
เกิดความเสียหายแก่บุคคลหรือทรัพย์สินของผู้อื่น
ไม่ว่าจะเป็นความผิดของตนหรือไม่ก็ตามต้องปฏิบัติดังนี้   -
ต้องหยุดรถหรือสัตว์และให้ความช่วยเหลือตามสมควรทันที   -
แสดงตัวและแจ้งเหตุต่อพนักงานเจ้าหน้าที่ใกล้เคียงทันที    -
ต้องแจ้งชื่อตัว ช...
Score:  1.000

{'page_label': '8', 'file_name': 'article_20160531140556.pdf', 'file_path': '/Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf', 'file_type': 'application/pdf', 'file_size': 301354, 'creation_date': '2024-07-29', 'last_modified_date': '2024-07-28'}
Node ID: 465c654b-4d12-4c77-807a-319699adc222
T

In [12]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode

query_enging = index.as_query_engine(vector_store_query_mode=VectorStoreQueryMode.HYBRID)

In [13]:
response_ = query_enging.query(question)
print(response_.response)

Context information is below.
---------------------
page_label: 8
file_path: /Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf

ข้อหา
หนึ่งรวมเป็น ๒ ข้อหา ความผิดฐานชนแล้วหนีมีหลักเกณฑ์ดังนี้  เมื่อขับรถหรือขี่หรือควบคุมสัตว์แล้วไปท าให้
เกิดความเสียหายแก่บุคคลหรือทรัพย์สินของผู้อื่น ไม่ว่าจะเป็นความผิดของตนหรือไม่ก็ตามต้องปฏิบัติดังนี้  
- ต้องหยุดรถหรือสัตว์และให้ความช่วยเหลือตามสมควรทันที  
- แสดงตัวและแจ้งเหตุต่อพนักงานเจ้าหน้าที่ใกล้เคียงทันที  
 - ต้องแจ้งชื่อตัว ชื่อสกุล และที่อยู่ของตน และเลขหมายทะเบียนรถแก่ผู้เสียหายด้วย  
ถ้าผู้ขับขี่หรือขี่รถควบคุมสัตว์หลบหนีไป กฎหมายให้สันนิฐานว่าเป็นผู้กระท าความผิดและ
พนักงานเจ้าหน้าที่มีอ

page_label: 8
file_path: /Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf

๘ 
 
ตั้งแต่ ๒,๐๐๐  บาท ถึง ๑๐,๐๐๐  บาท หรือทั้งจ าทั้งปรับ และให้ศาลสั่งพักใช้ใบอนุญาตขับขี่ของผู้นั้นมี
ก าหนดไม่น้อยกว่า ๑ เดือน หรือเพิกถอนใบอนุญาตขับขี่  
                ๙. กรณีชนแล้วหนี  
กฎหมายบัญญัติให้เป็นความผิดพิเศษเพิ่มขึ้นอีกก

In [14]:
prompt = prompt=response_.response

In [15]:
import json
import requests
url ='http://localhost:11434/api/generate'
payload = json.dumps({
    "model": "openthaigpt",
    "stream": False,
    "prompt": f'''<s>[INST] <<SYS>>
        You are a question answering assistant. Answer the question as truthful and helpful as possible
        คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด
        <</SYS>>

        Answer the question based only on the following context:
        {prompt}

        [/INST]''',
})
headers = {
'Content-Type': 'application/json'
}
response = requests.request("POST", url, headers=headers, data=payload)
response

<Response [200]>

In [16]:
if response.status_code == 200:
    response_text = response.text
    data = json.loads(response_text)
    actual_response = data["response"]
    print(actual_response)
else:
    print("Error:",response.status_code,response.text)

ไม่ว่าจะเกิดจากความผิดพลาดหรือละเลยไม่ยอมหยุดให้ความช่วยเหลือก็ตาม หากผู้ขับขี่หรือคนขับรถเกิดหลบหนีไป ก็จะได้รับผลในทางกฎหมายสำหรับความผิดฐาน "ขับรถชนแล้วหนี" ซึ่งตามกฎหมายมีหลักเกณฑ์ในการลงโทษอยู่ 2 ข้อ กล่าวคือ 
                **ข้อ 1. ข้อหาทำร้ายร่างกาย** ผู้ขับขี่จะโดนตั้งข้อหาว่าทำร้ายผู้บาดเจ็บสาหัส ตามกฎหมายอาญา มาตรา 340 ต้องระวางโทษจำคุกสูงสุด 5 ปี และปรับไม่เกิน 10,000 บาท 
                **ข้อ 2. ข้อหาหลบหนี** เมื่อมีการขับรถชนแล้วหนี ผู้ขับขี่จะต้องระวางโทษฐานกระทำการโดยประมาทตามประมวลกฎหมายอาญา มาตรา 396 ต้องระวางโทษจำคุกสูงสุด 18 เดือน และปรับไม่เกิน 5,000 บาท ซึ่งบทลงโทษนี้อาจนำไปประกอบกับมาตรา 297 และ 342 จะไม่ได้รับอนุญาตให้กระทำในระหว่างที่มีการประกันตัว หรือได้รับการปล่อยตัวชั่วคราว หรือไม่ได้ถูกจำคุกในคดีอาญาอื่น และในกรณีที่ผู้ขับขี่มีใบอนุญาตขับขี่หรือได้รับใบอนุญาตขับขี่แล้ว จะไม่ได้รับอนุญาตให้ใช้ใบอนุญาตนั้นตลอดชีวิต หากศาลตัดสินว่ามีความผิด 
นอกจากนี้ ผู้ขับขี่จะต้องชดใช้ค่าสินไหมทดแทนให้แก่ผู้บาดเจ็บด้วย


In [17]:
payload = f'''<s>[INST] <<SYS>>
        You are a question answering assistant. Answer the question as truthful and helpful as possible
        คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด
        <</SYS>>

        Answer the question based only on the following context:
        {prompt}

        [/INST]'''

In [18]:
import ollama
from ollama import Client
client = Client(host='http://localhost:11434')
stream = ollama.generate(
    model='openthaigpt',
    prompt=payload,
    stream=True,
)
for chunk in stream:
  print(chunk['response'], end='', flush=True)

จากบริบทที่ให้มา กฎหมายคุ้มครองผู้บาดเจ็บจากการขับรถ โดยระบุว่า "เมื่อขับรถหรือขี่หรือควบคุมสัตว์แล้วไปทำให้
เกิดความเสียหายแก่บุคคลหรือทรัพย์สินของผู้อื่น ไม่ว่าจะเป็นความผิดของตนหรือไม่ก็ตามต้องปฏิบัติดังนี้" ซึ่งหมายความว่าตามกฎหมาย ผู้ขับขี่จะต้องหยุดรถ/ควบคุมสัตว์ของตนเองหากทำให้บุคคลหรือทรัพย์สินของผู้อื่นเสียหาย เพื่อช่วยเหลือผู้บาดเจ็บและแจ้งเหตุไปยังเจ้าหน้าที่ตำรวจ ดังนั้น จากบริบทนี้ จึงสรุปได้ว่า "เมื่อขับรถหรือขี่หรือควบคุมสัตว์แล้วไปทำให้
เกิดความเสียหายแก่บุคคลหรือทรัพย์สินของผู้อื่น ไม่ว่าจะเป็นความผิดของตนหรือไม่ก็ตามต้องปฏิบัติดังนี้" ดังนั้น เมื่อขับรถชนแล้วหนี โดยทำให้บุคคลหรือทรัพย์สินของผู้อื่นเสียหาย ผู้ขับขี่จะต้องหยุดรถ/ควบคุมสัตว์และช่วยเหลือผู้บาดเจ็บ รวมถึงแจ้งเหตุไปยังเจ้าหน้าที่ตำรวจ ดังนั้นคำตอบที่ถูกต้องคือ 2. มีโทษฐานชนแล้วหนี